<a href="https://colab.research.google.com/github/JoyeBright/Multilingual_Multimodal_Universal_MT/blob/main/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
path = "/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Per_Eng_Trans_V1/raw_data/TEP"

## **Sentence-BERT**

In [2]:
import numpy as np

In [3]:
!pip3 install torch torchvision

In [4]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.3126, 0.9105, 0.8420],
        [0.7850, 0.4619, 0.3130],
        [0.0171, 0.4852, 0.2908],
        [0.5852, 0.3574, 0.5472],
        [0.0284, 0.4941, 0.6537]])


In [5]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.4.1.2)


In [14]:
train_src = path+"/train.tok.clean.src"

In [15]:
!head -10 $train_src

یون سنگ، تو میری از او بپرسی چیزی پیدا کرده یا نه؟
چيز غيرعادي اي نبود .
...البته کاملا مزه گوشت گاو را نداره
شماها باید به این خاتمه بدین .
بله. اما او به نظر آشفته بود. مشکلی پیش اومده؟
ولي خوني شده .
شيطنت کردن .
اما اگر عميقتر ببينيد برتر از اين هم وجود داره، يعني زير زمين
معنی سادش اینه که شما میخواید فرار کنید .
باقيماندن .


In [16]:
from sentence_transformers import SentenceTransformer
name = 'https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/xlm-r-100langs-bert-base-nli-mean-tokens.zip'

In [17]:
model = SentenceTransformer(model_name_or_path=name, device='cuda')

In [18]:
print("Max Sequence Length:", model.max_seq_length)

Max Sequence Length: 128


In [19]:
with open(train_src) as f:
    content = f.readlines()
    content = [x.strip() for x in content]

In [20]:
sentences = content

In [21]:
sentence_embeddings = model.encode(sentences=sentences, show_progress_bar=True, convert_to_numpy=True)

In [22]:
print(type(sentence_embeddings))
print(sentence_embeddings.shape)

<class 'numpy.ndarray'>
(610086, 768)


In [11]:
import pickle

In [24]:
#Store sentences & embeddings on disc
with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': sentences, 'embeddings': sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
#Load sentences & embeddings from disc
with open('/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Domain_Adaptation/SentenceBERT/embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

In [13]:
print(stored_embeddings.shape)
print(type(stored_embeddings))

(610086, 768)
<class 'numpy.ndarray'>


In [14]:
samples = 130000

In [15]:
stored_embeddings = stored_embeddings[:samples]
stored_sentences = stored_sentences[:samples]

print(stored_embeddings.shape)

(130000, 768)


In [16]:
print(stored_embeddings[:1])

[[ 0.4593516  -0.01457303  1.7394984   0.4404026   0.6362638   0.2528962
   0.11195075  0.2921515   0.10742761  0.15947405 -0.21293342  0.5458668
   0.2740503   0.5479267  -0.32321924  0.13399549  0.04454777 -0.57006615
   0.18741901 -0.5034281   0.17253752  0.3391332   0.10633039 -0.69341135
   0.30869687 -0.06997975 -0.12753329 -0.82400626 -0.6240134   0.20505981
   0.03349856 -0.04983833 -0.27879694 -0.17779048 -0.898158    0.9478081
   0.7969705   0.10409191  0.04230267  0.0060135   0.0042068   0.06179347
   0.30250612  0.8499539  -0.9323933  -0.5255916   0.36498743  0.12332773
  -0.19779778 -1.1141446  -0.5156098   0.08481535  0.55320185  0.39266366
  -0.669689   -0.03589565  0.5764537  -1.7097796  -0.14665571  0.27364424
  -1.2197844  -0.17900002  0.81105083  0.49735194 -0.09490854  0.6097086
   0.13635202 -0.41592193 -0.5177906  -0.04319844  0.6383545  -0.5826326
  -0.08961103  0.6948251  -0.37232247 -0.10304417  0.00706872  0.81251115
   0.48310867  0.6950191  -0.22152555  0.08

In [9]:
!pip install gpustat

In [10]:
!gpustat -cp

57e15a3dbd9d         Sat Feb  6 15:23:44 2021  418.67
[0] Tesla T4         | 61'C,   0 % |     0 / 15079 MB |


In [11]:
from sentence_transformers import util

## **Clustering using Sentence-Transformers package**

In [12]:
def community_detection(embeddings, threshold=0.75, min_community_size=10, init_max_size=1000):
    """
    Function for Fast Community Detection
    Finds in the embeddings all communities, i.e. embeddings that are close (closer than threshold).
    Returns only communities that are larger than min_community_size. The communities are returned
    in decreasing order. The first element in each list is the central point in the community.
    """

    # Compute cosine similarity scores
    cos_scores = util.pytorch_cos_sim(embeddings, embeddings)

    # Minimum size for a community
    top_k_values, _ = cos_scores.topk(k=min_community_size, largest=True)

    # Filter for rows >= min_threshold
    extracted_communities = []
    for i in range(len(top_k_values)):
        if top_k_values[i][-1] >= threshold:
            new_cluster = []

            # Only check top k most similar entries
            top_val_large, top_idx_large = cos_scores[i].topk(k=init_max_size, largest=True)
            top_idx_large = top_idx_large.tolist()
            top_val_large = top_val_large.tolist()

            if top_val_large[-1] < threshold:
                for idx, val in zip(top_idx_large, top_val_large):
                    if val < threshold:
                        break

                    new_cluster.append(idx)
            else:
                # Iterate over all entries (slow)
                for idx, val in enumerate(cos_scores[i].tolist()):
                    if val >= threshold:
                        new_cluster.append(idx)

            extracted_communities.append(new_cluster)

    # Largest cluster first
    extracted_communities = sorted(extracted_communities, key=lambda x: len(x), reverse=True)

    # Step 2) Remove overlapping communities
    unique_communities = []
    extracted_ids = set()

    for community in extracted_communities:
        add_cluster = True
        for idx in community:
            if idx in extracted_ids:
                add_cluster = False
                break

        if add_cluster:
            unique_communities.append(community)
            for idx in community:
                extracted_ids.add(idx)

    return unique_communities

In [ ]:
import time
print("Start clustering")
start_time = time.time()
clusters = community_detection(stored_embeddings, min_community_size=40, threshold=0.95)
#Print all cluster / communities
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster:
        print("\t", stored_sentences[sentence_id])

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

## **Top2Vec**

In [10]:
!pip install top2vec[sentence_transformers]

     |████████████████████████████████| 307kB 18.7MB/s 
     |████████████████████████████████| 6.4MB 23.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp36-cp36m-linux_x86_64.whl size=2311380 sha256=7a0ff6ced384420af60da0b51886742e83c5c9a2ed0f9c43eb9361dcd3e4246d
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan
ERROR: hdbscan 0.8.27 has requirement joblib>=1.0, but you'll have joblib 0.17.0 which is incompatible.
ERROR: top2vec 1.0.21 has requirement numpy==1.19.2, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: joblib 1.0.0
    Uninstalling joblib-1.0.0:
      Successfully uninstalled joblib-1.0.0


In [13]:
from top2vec import Top2Vec

In [24]:
model = Top2Vec(stored_sentences[:samples], embedding_model='distiluse-base-multilingual-cased')

2021-02-06 15:25:25,802 - top2vec - INFO - Pre-processing documents for training
2021-02-06 15:25:26,095 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model
2021-02-06 15:25:43,620 - top2vec - INFO - Creating joint document/word embedding
2021-02-06 15:25:55,677 - top2vec - INFO - Creating lower dimension embedding of documents
2021-02-06 15:26:34,241 - top2vec - INFO - Finding dense areas of documents
2021-02-06 15:26:34,688 - top2vec - INFO - Finding topics


In [30]:
print("Number of topics:", model.get_num_topics())


Number of topics: 96


In [34]:
topic_sizes, topic_nums = model.get_topic_sizes()
print("topic size", topic_sizes)
print("topic number", topic_nums)

topic size [803 390 315 264 260 223 211 201 199 196 195 191 171 165 161 157 143 142
 141 141 133 130 129 122 118 116 116 113 108 105 105 105 101  98  95  92
  91  91  90  88  87  86  85  85  85  84  83  81  80  79  78  75  74  73
  73  73  66  65  65  64  64  63  62  62  62  61  59  58  56  54  53  53
  52  52  48  48  48  44  44  42  42  42  42  41  41  39  38  36  36  33
  31  30  30  28  27  23]
topic number [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]


In [41]:
topic_words, word_scores, topic_nums = model.get_topics(3)

In [42]:
print(topic_words)

[['شد' 'پيش' 'کردن' 'مورد' 'به' 'می' 'ولي' 'روي' 'را' 'توي' 'هر' 'كن'
  'خاطر' 'يک' 'شده' 'از' 'کار' 'يه' 'بيرون' 'كار' 'بهش' 'رو' 'انجام'
  'حرف' 'اي' 'يك' 'هستند' 'او' 'کي' 'دست' 'در' 'سر' 'يا' 'اقاي' 'کرد'
  'بده' 'راه' 'مي' 'ما' 'دوست' 'كه' 'کن' 'هم' 'که' 'کني' 'بوده' 'منو'
  'کنيد' 'اما' 'نفر']
 ['کنم' 'ميکنم' 'ميخوام' 'کردم' 'كنم' 'دارم' 'هستم' 'ميتونم' 'ميشه'
  'لطفا' 'بودم' 'بي' 'بهت' 'بده' 'با' 'منو' 'کنيم' 'گفتم' 'بر' 'بيا'
  'شده' 'برو' 'مي' 'ميخواي' 'کني' 'داشته' 'بهش' 'کن' 'ميکني' 'اونو' 'يا'
  'بوده' 'کنيد' 'تو' 'روي' 'فکر' 'يك' 'توي' 'خاطر' 'باش' 'رو' 'تمام'
  'هستي' 'اقاي' 'يک' 'اوه' 'به' 'شما' 'بيرون' 'می']
 ['ميخواي' 'شده' 'اونا' 'اونها' 'هستند' 'روي' 'بده' 'اوه' 'ها' 'بر'
  'اقاي' 'هم' 'ميدوني' 'برو' 'کردن' 'لطفا' 'کنيد' 'توي' 'هاي' 'کرد'
  'کرده' 'سر' 'ميکني' 'بچه' 'کني' 'با' 'بهت' 'بيا' 'يا' 'بهش' 'براي'
  'ولي' 'کي' 'يک' 'اينجا' 'پيش' 'دست' 'خب' 'صبر' 'هر' 'مرد' 'سلام' 'كه'
  'اونجا' 'کنيم' 'نگاه' 'شما' 'به' 'می' 'شد']]


In [59]:
words, word_scores = model.similar_words(keywords=["خاطر"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

شد 0.8685322168532293
به 0.8597604432911965
را 0.849143525509674
هر 0.8305449815171645
می 0.8293873189723151
پيش 0.8251956479906721
كن 0.8215255084392645
ولي 0.8194528472663456
مورد 0.8136457654968112
رو 0.812031051290705
او 0.8035998871277281
بهش 0.799646851987762
کردن 0.7995370442710003
يك 0.7971300720991035
انجام 0.7921365155972069
روي 0.7919249168401934
شده 0.7868035389615263
يک 0.7866085826817532
مي 0.7851309341018387
توي 0.7813211966054003


# **BERTopic**

In [17]:
!pip install bertopic

In [18]:
!pip install bertopic[visualization]

In [19]:
from bertopic import BERTopic

In [20]:
model = BERTopic(language="persian", verbose=True, min_topic_size = 200, calculate_probabilities = False)

In [21]:
topics, probabilities = model.fit_transform(stored_sentences)

2021-02-06 19:48:37,275 - BERTopic - Loaded embedding model
2021-02-06 19:51:16,044 - BERTopic - Transformed documents to Embeddings
2021-02-06 19:54:30,432 - BERTopic - Reduced dimensionality with UMAP
2021-02-06 19:55:05,703 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-02-06 19:56:23,840 - BERTopic - Loaded embedding model
2021-02-06 19:56:24,494 - BERTopic - Transformed documents to Embeddings


In [22]:
model.save("/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Domain_Adaptation/BERTopic/BERTopic-130000")

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [39]:
print(model.get_topic_freq().head(12))

    Topic  Count
0      -1  56071
1      56  12370
2      58   6373
3      66   6075
4      60   5338
5      35   5277
6      55   4044
7      52   2566
8      53   2255
9      18   1497
10     61   1479
11     23   1402


In [24]:
print(model.get_topic_freq())

    Topic  Count
0      -1  56071
1      56  12370
2      58   6373
3      66   6075
4      60   5338
..    ...    ...
65      5    214
66      1    209
67     14    208
68     39    204
69     48    200

[70 rows x 2 columns]


In [28]:
print(np.asarray(model.get_topic(56)))

[['نيست' '0.05977586894407984']
 ['هيچ' '0.050257761219075645']
 ['نمي' '0.03751253834723982']
 ['نداره' '0.021248006731086656']
 ['نميتونم' '0.020253333644432502']
 ['نبايد' '0.019192535271702917']
 ['ندارم' '0.018632407674427767']
 ['نيستم' '0.01720129287169697']
 ['نميدونم' '0.01684624676224587']
 ['نبود' '0.01666859699460104']]


In [30]:
print(np.asarray(model.get_topic(58)))

[['در' '0.024744293114542584']
 ['خانم' '0.02446310353877173']
 ['آقاي' '0.023052635995883943']
 ['وايسا' '0.022167231439952872']
 ['کننده' '0.01623433920320345']
 ['زدن' '0.01393350594100566']
 ['سازي' '0.012678505850039731']
 ['اسم' '0.012270066072090236']
 ['هستم' '0.012123523474977301']
 ['سارا' '0.010737950240098418']]


In [31]:
print(np.asarray(model.get_topic(66)))

[['من' '0.07192896316340686']
 ['همه' '0.03900659812610119']
 ['بله' '0.036903665032047596']
 ['آره' '0.03606870195203992']
 ['بايد' '0.03535125306043873']
 ['ميدونم' '0.023732514211508366']
 ['کنم' '0.022150794536632965']
 ['کردم' '0.01903608180721355']
 ['کار' '0.01891553828365745']
 ['دارم' '0.01789114717834658']]


In [32]:
print(np.asarray(model.get_topic(60)))

[['دوست' '0.059950179962113444']
 ['ممنون' '0.05420529400640519']
 ['ممنونم' '0.040799489880713635']
 ['با' '0.02962781913952386']
 ['خوبي' '0.028571496501096295']
 ['خوشحالم' '0.028548020605381']
 ['خوش' '0.025490455213783868']
 ['خوشحال' '0.022360925179307243']
 ['خوبه' '0.021779400194036923']
 ['متشکرم' '0.018789323847960577']]


In [34]:
print(np.asarray(model.get_topic(35)))

[['چي' '0.11893377447998849']
 ['کجا' '0.06973738717995064']
 ['چرا' '0.061890756622314615']
 ['چيه' '0.05683784799710331']
 ['چطور' '0.05674554903874951']
 ['چی' '0.0461096898398823']
 ['کجاست' '0.030409708909152802']
 ['كجا' '0.028128859299020696']
 ['چيکار' '0.02583106299984453']
 ['چکار' '0.02438472956376564']]


In [35]:
print(np.asarray(model.get_topic(55)))

[['اشتباه' '0.03914418456097871']
 ['بد' '0.03477199024527016']
 ['در' '0.03003491725319932']
 ['احمق' '0.024605887807324853']
 ['ما' '0.02191918805863311']
 ['شده' '0.019105517152166133']
 ['مشکل' '0.01644292850360572']
 ['بدي' '0.016350326752024538']
 ['سخت' '0.015560514092856942']
 ['عصباني' '0.015361646891762046']]


In [36]:
print(np.asarray(model.get_topic(52)))

[['بيرون' '0.057895262397405785']
 ['رو' '0.053063753100471636']
 ['فرار' '0.05139678257147021']
 ['برو' '0.034324613453656005']
 ['گم' '0.028947714104915142']
 ['فراموش' '0.027656160929691514']
 ['شو' '0.025308137018134176']
 ['کنار' '0.02357028205736253']
 ['عقب' '0.01988031789068785']
 ['خارج' '0.017938849196546338']]


In [37]:
print(np.asarray(model.get_topic(53)))

[['مرده' '0.0891447234411005']
 ['مرگ' '0.07832660162074728']
 ['کشته' '0.04969578448820682']
 ['قتل' '0.039334228912873456']
 ['قاتل' '0.030393329099010404']
 ['کشتن' '0.02463882358779625']
 ['منو' '0.022196465558109175']
 ['مردن' '0.021545649284491458']
 ['انتقام' '0.02105202349554633']
 ['بکشه' '0.019517592062294666']]


In [38]:
print(np.asarray(model.get_topic(18)))

[['من' '0.046895723469860304']
 ['معامله' '0.03630191251769074']
 ['در' '0.03466773582252822']
 ['اين' '0.034057600422382484']
 ['تجارت' '0.029965448299794147']
 ['بازار' '0.0249870680843062']
 ['ثروت' '0.021794032028268347']
 ['پرداخت' '0.01754906714912618']
 ['شما' '0.017462265608244883']
 ['بانک' '0.016045116848202614']]


In [40]:
print(np.asarray(model.get_topic(61)))

[['بچه' '0.1377755079527332']
 ['پدر' '0.10827110187727192']
 ['پسر' '0.09637317859405523']
 ['برادر' '0.06492388361188059']
 ['بچهها' '0.06326816683358519']
 ['پدرت' '0.04976964652262224']
 ['پدرم' '0.048470135677557244']
 ['بابا' '0.0444886225098586']
 ['پسرم' '0.032462412988574496']
 ['برادرم' '0.031589357381086494']]


In [41]:
print(np.asarray(model.get_topic(23)))

[['فردا' '0.10161074356667975']
 ['روز' '0.08406525424260831']
 ['هفته' '0.0599507237170682']
 ['ساعت' '0.05467172138899506']
 ['زمان' '0.043814450829318195']
 ['وقت' '0.038468670009476125']
 ['امروز' '0.03577433133869311']
 ['سال' '0.03055498198228443']
 ['آينده' '0.030465502828381792']
 ['ديروز' '0.026793289866740212']]


In [26]:
model.visualize_topics()